In [1]:
import pandas as pd
import xarray as xr
import xoak
import tqdm
from pathlib import Path

In [2]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:36779")
client

<Client: 'tcp://127.0.0.1:36779' processes=8 threads=32, memory=46.88 GiB>

In [3]:
catalog = pd.read_csv("/home/jovyan/shared_materials/climattr_data_please-dont-copy/nc/catalog.csv")
catalog

,domain_resolution,gcm,rcm,variable,data_file_glob
0,AFR-22,CCCma-CanESM2,CCCma-CanRCM4,pr,/home/jovyan/shared_materials/climattr_data_pl...
1,AFR-22,MOHC-HadGEM2-ES,CLMcom-KIT-CCLM5-0-15,pr,/home/jovyan/shared_materials/climattr_data_pl...
2,AFR-22,MOHC-HadGEM2-ES,GERICS-REMO2015,pr,/home/jovyan/shared_materials/climattr_data_pl...
3,AFR-22,MPI-M-MPI-ESM-LR,CLMcom-KIT-CCLM5-0-15,pr,/home/jovyan/shared_materials/climattr_data_pl...
4,AFR-22,MPI-M-MPI-ESM-LR,GERICS-REMO2015,pr,/home/jovyan/shared_materials/climattr_data_pl...
5,AFR-22,NCC-NorESM1-M,CLMcom-KIT-CCLM5-0-15,pr,/home/jovyan/shared_materials/climattr_data_pl...
6,AFR-22,NCC-NorESM1-M,GERICS-REMO2015,pr,/home/jovyan/shared_materials/climattr_data_pl...
7,EUR-11,CCCma-CanESM2,CLMcom-CCLM4-8-17,pr,/home/jovyan/shared_materials/climattr_data_pl...
8,EUR-11,CNRM-CERFACS-CNRM-CM5,CNRM-ALADIN63,pr,/home/jovyan/shared_materials/climattr_data_pl...
9,EUR-11,IPSL-IPSL-CM5A-MR,DMI-HIRHAM5,pr,/home/jovyan/shared_materials/climattr_data_pl...


In [ ]:
for nrun in tqdm.tqdm(range(len(catalog))):
    
    ds = xr.open_mfdataset(
        catalog.iloc[nrun].data_file_glob,
        chunks={"time": -1, "rlon": 30, "rlat": 30},
        decode_cf=False,
    )
    if "rlon" not in ds.dims:
        ds = xr.open_mfdataset(
            catalog.iloc[nrun].data_file_glob,
            chunks={"time": -1, "x": 30, "y": 30},
            decode_cf=False,
        )   
    ds = ds.assign_coords(
        lon=ds.lon.isel(time=0, drop=True).compute(),
        lat=ds.lat.isel(time=0, drop=True).compute(),
    )
    ds = ds.chunk({"time": -1})
    ds = ds.assign_coords(
        time=xr.decode_cf(ds[["time"]]).time.compute(),
    )
    ds = ds.drop_vars([v for v in ds.data_vars.keys() if v not in ["pr", "tasmax"]])
    ds = ds.assign_coords(**dict(catalog.iloc[nrun]))
    
    outname = (
        f"{list(ds.data_vars.keys())[0]}"
        f"_{ds.domain_resolution.data[()]}"
        f"_{ds.gcm.data[()]}"
        f"_{ds.rcm.data[()]}"
        ".zarr"
    )

    try:
        ds.drop_encoding().to_zarr(
            Path("/home/jovyan/shared_materials/climattr_data_please-dont-copy/zarr/") / outname,
        )
    except:
        print(f"skipping {outname}")
        pass
    print(outname, "done")

  3%|▎         | 1/31 [00:13<06:46, 13.56s/it]

skipping pr_AFR-22_CCCma-CanESM2_CCCma-CanRCM4.zarr
pr_AFR-22_CCCma-CanESM2_CCCma-CanRCM4.zarr done


  6%|▋         | 2/31 [00:15<03:07,  6.46s/it]

skipping pr_AFR-22_MOHC-HadGEM2-ES_CLMcom-KIT-CCLM5-0-15.zarr
pr_AFR-22_MOHC-HadGEM2-ES_CLMcom-KIT-CCLM5-0-15.zarr done
